In [1]:
import numpy, scipy

In [2]:
from scipy.optimize import minimize

##$\mathord{\rm CO}_2$ $\mathord{\rm N}_2$ example

In [3]:
from math import log

In [4]:
mdCO2 = 1.0
mdN2 = 2.5
mmCO2 = 12 + 2 * 16
mmN2 = 2 * 14
pCO2 = 1.5
pN2 = 1.0
p = 0.7
TCO2 = 35.0
TN2 = 150

cpN2 = 0.8485
cpCO2 = 1.0366

###Temperature

In [5]:
md = mdCO2 + mdN2; md

3.5

In [6]:
wCO2 = mdCO2/md
wN2 = mdN2/md
wCO2, wN2

(0.2857142857142857, 0.7142857142857143)

In [7]:
assert wCO2 + wN2 == 1

In [8]:
cp = cpCO2 * wCO2 + cpN2 * wN2; cp

0.9022428571428571

In [9]:
T = (cpCO2*wCO2*TCO2 + cpN2*wN2 * TN2)/cp; T

112.24994854093767

###Entropy variation

In [10]:
nCO2 = mdCO2/mmCO2
nN2 = mdN2/mmN2
n = nN2 + nCO2
nCO2, nN2

(0.022727272727272728, 0.08928571428571429)

In [11]:
xCO2 = nCO2/n
xN2 = nN2/n
xCO2, xN2

(0.2028985507246377, 0.7971014492753624)

In [12]:
assert xN2 + xCO2 == 1

In [13]:
pCO22 = xCO2 * p
pN22 = xN2 *p 
print pCO22, pN22
R = 8.314

0.142028985507 0.557971014493


In [14]:
assert abs(pCO22 + pN22 - p) < 1e-7

In [15]:
toKelvin = lambda T: T + 273

In [16]:
# T=112. a les transperències. Per què?
dSCO2 = cpCO2 * log(toKelvin(T)/toKelvin(TCO2)) - R/mmCO2 * log(pCO22/pCO2)
dSN2 = cpN2 * log(toKelvin(T)/toKelvin(TN2)) - R/mmN2 * log(pN22/pN2)
print dSCO2, dSN2
dS = mdCO2 * dSCO2 + mdN2 * dSN2; dS

0.677384984207 0.0939248164265


0.9121970252733903

###Same temperature

In [17]:
cpN2, 8.314/44

(0.8485, 0.18895454545454546)

In [18]:
from scipy import optimize

In [19]:
optimize?

In [20]:
class Species:
    def __init__(self, **kw):
        self.kw = kw
        
    def _get(self,k):
        from numpy import NaN
        return self.kw.get(k, NaN)
                
    def __getitem__(self,k):
        return self._get(k)
    
    @property
    def sum(self,k):
        return sum(self._get(k))
    
    @property
    def Rg(self):
        return 8.314/self['mol']
    
    
class Mixture:
    def __init__(self, *sps):
        self.species = sps
        
    def __getitem__(self,f):
        from numpy import array, double
        return array([s[f] for s in self.species], dtype=double)
    

In [37]:
CO2 = Species(mol=14+2*16, cp=1.0366, md=1.0, p=1.5, T=35)
N2 = Species(mol=2*14, cp=0.8485 , md=2.5, p=1, T=150)
mix = Mixture(CO2, N2)

In [38]:
w = mix['md']/sum(mix['md']); w

array([ 0.28571429,  0.71428571])

In [39]:
mix['T'].dot(w)/mix['cp'].dot(w)

129.83517266494607

In [40]:
w.dot(mix['cp'] * mix['T'])/w.dot(mix['cp'])

112.24994854093767